In [28]:
from lxml import html
import requests
import unicodecsv as csv
import argparse
import json
import requests
import bs4
import pandas as pd
import time

In [32]:
attrs_of_interest=['list-card-addr','list-card-type','list-card-price','list-card-details']
data={}
for page in range(1,5):
    time.sleep(1)
    url=f"https://www.zillow.com/lewisburg-pa-17837/{page}_p"
    print(url)
    resp = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(resp.text)
    data_len=len(data)
    for i,x in enumerate(soup.find_all(attrs={'class': ['list-card']})):
        data[data_len+i]={}
        for y in x.find_all(attrs={'class': ['list-card-info']}):
            for z in y.find_all(attrs={'class': attrs_of_interest}):
                data[data_len+i][x.attrs['class'][0]]=x.text
                data[data_len+i][y.attrs['class'][0]]=y.text
                data[data_len+i][z.attrs['class'][0]]=z.text

https://www.zillow.com/lewisburg-pa-17837/1_p
https://www.zillow.com/lewisburg-pa-17837/2_p
https://www.zillow.com/lewisburg-pa-17837/3_p
https://www.zillow.com/lewisburg-pa-17837/4_p


In [33]:
result = pd.DataFrame.from_dict(data,orient='index')

In [34]:
result.to_csv("properties-17837.csv")